In [188]:
# Disable jedi autocompleter
%config Completer.use_jedi = False

In [189]:
#imports
import pandas as pd
columns_to_extract=['App_ID','State','CDO','LOB']
key_primary=['App_ID','CDO']
key_column_changes_to_track=['State','CDO','LOB']

#files:
input_file=r"C:\\Users\\John\\Downloads\\[FreeCourseSite.com] Udemy - The Complete Python Course  Learn Python by Doing\\1. Intro to Python\\delete_input.xlsx"
x=','.join(key_primary)
x
#my_array = [phrase.split() for phrase in names]
#df[['First Name New', 'Last Name New']] = df['Name'].str.split(' ', expand=True)
#https://datagy.io/create-new-columns-in-pandas/
#TODO : duplicate the important columns into previous and current beforehand rather than post-join

'App_ID,CDO'

In [190]:
#jupyter settings
#pd.describe_option()
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

#??pd.read_excel

In [191]:
#UDF
def columns_for_join(df:pd.DataFrame,text_identifier:str)->list:
    col_prev=[]
    for i in df.columns:
        if i in key_primary:
            col_prev.append(i)
        elif i.find(text_identifier)>0:
            col_prev.append(i)
    return col_prev

In [192]:
#raw data
df_previous=pd.read_excel(input_file,sheet_name='previous',usecols=columns_to_extract,dtype={'App_ID':'int'})
df_current=pd.read_excel(input_file,sheet_name='current',usecols=columns_to_extract,dtype={'App_ID':'int'})

#suffixes: 'Suffixes' = ('_x', '_y'),
#indicator: 'bool' = False,
df_current.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   App_ID  5 non-null      int32 
 1   State   5 non-null      object
 2   CDO     5 non-null      object
 3   LOB     5 non-null      object
dtypes: int32(1), object(3)
memory usage: 268.0+ bytes


In [193]:
df=pd.merge(df_previous,df_current,on=key_primary,how='outer',suffixes=('_previous', '_current'),indicator=True)
df=df.query("_merge != 'both'") #no changes
df

,App_ID,State_previous,CDO,LOB_previous,State_current,LOB_current,_merge
0,100,Operate,CIB,CIB,NaN,NaN,left_only
2,300,Retired,CCB,CCB,NaN,NaN,left_only
4,500,Retired,CCB,CCB,NaN,NaN,left_only
5,100,NaN,CTC,NaN,Operate,CIB,right_only
6,300,NaN,CIB,NaN,Operate,CCB,right_only
7,500,NaN,CIB,NaN,Retired,CIB,right_only


In [194]:
#for rows which have changed evaluate earlier and current values
#get the previous rows for changed rows
col=columns_for_join(df,'_previous')
df2=df.query("_merge =='left_only'")[col]
df2=df2.rename(columns={'CDO':'CDO_previous'})
df2



,App_ID,State_previous,CDO_previous,LOB_previous
0,100,Operate,CIB,CIB
2,300,Retired,CCB,CCB
4,500,Retired,CCB,CCB


In [195]:
#for rows which have changed evaluate earlier and current values
#get the current rows for changed rows
col=columns_for_join(df,'_current')
df3=df.query("_merge =='right_only'")[col]
df3=df3.rename(columns={'CDO':'CDO_current'})
df3


,App_ID,CDO_current,State_current,LOB_current
5,100,CTC,Operate,CIB
6,300,CIB,Operate,CCB
7,500,CIB,Retired,CIB


In [196]:
#for the rows that have changed get the previous and current columns
df4=pd.merge(df2,df3,on='App_ID',how='outer')
df4=df4
df4

,App_ID,State_previous,CDO_previous,LOB_previous,CDO_current,State_current,LOB_current
0,100,Operate,CIB,CIB,CTC,Operate,CIB
1,300,Retired,CCB,CCB,CIB,Operate,CCB
2,500,Retired,CCB,CCB,CIB,Retired,CIB


In [197]:
#create boolean flag columns
#condition 1 : TODO : only include current operate apps which are in Operate
df4.loc[(df4['CDO_current']=='CIB') & (df4['State_current']=='Operate') & (df4['State_previous']!='Operate'),'Reason']='Moved to CIB CDO due to move to Operate'
df4.loc[(df4['CDO_current']=='CIB') & (df4['LOB_current']=='CIB') & (df4['LOB_previous']!='CIB'),'Reason']='Moved to CIB CDO due to LOB change to CIB'
df4

,App_ID,State_previous,CDO_previous,LOB_previous,CDO_current,State_current,LOB_current,Reason
0,100,Operate,CIB,CIB,CTC,Operate,CIB,NaN
1,300,Retired,CCB,CCB,CIB,Operate,CCB,Moved to CIB CDO due to move to Operate
2,500,Retired,CCB,CCB,CIB,Retired,CIB,Moved to CIB CDO due to LOB change to CIB
